## First try

In [ ]:
# !pip install torch torchvision

In [5]:
!python --version

Python 3.10.19


In [21]:
# imports 
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import f1_score

In [3]:
# define the device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


#### Handle input

In [12]:
# define directory
data_dir = "./data"

# transform to convert input into pytorch tensor
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# train and test data 
train_dataset = datasets.ImageFolder(root = os.path.join(data_dir, "train"), transform = transform)
test_dataset  = datasets.ImageFolder(root = os.path.join(data_dir, "test"), transform = transform)

# data loader objects for efficient batch processing during training
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
X_train = []
y_train = []
for X, y in train_loader:
    X_train.append(X)
    y_train.append(y)
# concatenate into one tensor
X_train = torch.cat(X_train, dim = 0)
y_train = torch.cat(y_train, dim = 0)

X_test = []
y_test = []
for X, y in test_loader:
    X_test.append(X)
    y_test.append(y)
# concatenate into one tensor
X_test = torch.cat(X_test, dim = 0)
y_test = torch.cat(y_test, dim = 0)

#### Defining the class of the DNN

In [17]:
# define input dims and num classes
input_dim = 224 * 224 * 3
num_classes = len(train_dataset.classes)

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # flatten input
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [19]:
# 
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150528, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=4, bias=True)
  )
)


**Architecture**: Linear(150528 → 512) → ReLU → Linear(512 → 256) → ReLU → Linear(256 → 4)

#### Train

In [20]:
# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 10.4570
Epoch [20/100], Loss: 6.4017
Epoch [30/100], Loss: 2.3250
Epoch [40/100], Loss: 2.2060
Epoch [50/100], Loss: 1.3950
Epoch [60/100], Loss: 1.2618
Epoch [70/100], Loss: 1.2471
Epoch [80/100], Loss: 1.2057
Epoch [90/100], Loss: 1.1948
Epoch [100/100], Loss: 1.1814


In [24]:
with torch.no_grad():
    train_preds = model(X_train).argmax(dim = 1).cpu().numpy()
    train_labels = y_train.cpu().numpy()
    train_f1 = f1_score(train_labels, train_preds, average = "macro")

    test_preds = model(X_test).argmax(dim = 1).cpu().numpy()
    test_labels = y_test.cpu().numpy()
    test_f1 = f1_score(test_labels, test_preds, average = "macro")

print(f"Train F1: {train_f1} \nTest F1: {test_f1}")

Train F1: 0.4806553612953588 
Test F1: 0.4860035871349533


The results are bad, the training f1 is low so there is a high bias so we need to fix it by:
- Adding more layers
- Adding more neurons
- Changing the activation functions
- Changing the weight initialization